# Search for collections with CMR search

In [1]:
import requests
from xml.dom.minidom import parse, parseString

## Find all collections
This request returns all collections with no filtering.

In [2]:
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections")
print(response.headers['CMR-Hits'] + " hits")

28347 hits


## Find a collection by concept id
We can add a concept id constraint to the search to get the collection matching that concept id.

A CMR concept id is in the format `<concept-type-prefix> <unique-number> "-" <provider-id>`

* `concept-type-prefix` is a single capital letter prefix indicating the concept type. "C" is used for collections
* `unique-number` is a single number assigned by the CMR during ingest.
* `provider-id` is the short name for the provider. i.e. "LPDAAC_ECS"

In [3]:
content_id = "C1934541400-SCIOPS"
response = requests.get("https://cmr.earthdata.nasa.gov/search/collections?concept_id=" + content_id)
print("The following collection has the content_id " + content_id + ":\n")
pretty = parseString(response.content).toprettyxml()
print(pretty)

The following collection has the content_id C1934541400-SCIOPS:

<?xml version="1.0" ?>
<results>
	<hits>1</hits>
	<took>80</took>
	<references>
		<reference>
			<name>&quot;The Omnivores Dilemma&quot;: The Effect of Autumn Diet on Winter Physiology and Condition of Juvenile Antarctic Krill</name>
			<id>C1934541400-SCIOPS</id>
			<location>https://cmr.earthdata.nasa.gov:443/search/concepts/C1934541400-SCIOPS/1</location>
			<revision-id>1</revision-id>
		</reference>
	</references>
</results>

